In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
import openai
import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from GptCOTPrompter import *
from GptCOTPrompter_BeamSeach import *
from AutoReasoner import *
import dotenv

config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY']

dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')
# dataset = pd.read_csv('./dataset/WikiTableQuestions/data/training.tsv', sep='\t')

In [3]:
import fasttext
ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
# change the majority vote to max score

NNDemo = False
max_demo = 5
template = 'original-sql-py'
# template = 'formatv1-sql-py'
def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_BeamSeach(
                                              f'prompt_template/{template}.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'],  
                                              base_path='./dataset/WikiTableQuestions/',
                                              demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                             )
            codex_prompter.max_demo = max_demo
            # codex_prompter._gen_gpt_prompt()
            codex_prompter._gen_gpt_prompt(NNDemo, ft)
            codex_prompter._get_gpt_prediction(beam_size=5)
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            if "model's maximum context length" in str(e):
                return log
            max_retry -= 1
    return log
    
for program in [ 'sql-py']:
    n_threads = 10
    maxLimit = float('inf')
    # maxLimit = 200
    from joblib import Parallel, delayed
    output_result_file = f'./dataset/WikiTableQuestions/results/CodexAnswerCOTExecutor_BeamSeach_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables.json'
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
    json.dump(logs, open(output_result_file, 'w'), indent=4)
    # evaluate: 
    os.system(f'cd ./dataset/WikiTableQuestions/ && python2 evaluator.py ./results/{output_result_file.split("/")[-1]} && cd ..')

  1%|██                                                                                                                                                                                                                                                                                                             | 30/4344 [00:08<22:28,  3.20it/s]

This model's maximum context length is 8193 tokens, however you requested 8222 tokens (7198 in your prompt; 1024 for the completion). Please reduce your prompt; or completion length.


  2%|████▊                                                                                                                                                                                                                                                                                                        | 70/4344 [00:54<1:08:04,  1.05it/s]

This model's maximum context length is 8193 tokens, however you requested 8649 tokens (7625 in your prompt; 1024 for the completion). Please reduce your prompt; or completion length.


  3%|████████▉                                                                                                                                                                                                                                                                                                   | 130/4344 [01:59<1:23:36,  1.19s/it]/mnt/GPT-TabQA/GptCOTPrompter.py:47: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  renewed_df = ps.sqldf(code)
/mnt/GPT-TabQA/GptCOTPrompter.py:47: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  renewed_df = ps.sqldf(code)
  3%|██████████▎                                                                                                                                                                                                                                                                        

...

 26%|████████████████████████████████████████████████████████████████████████████▍                                                                                                                                                                                                                              | 1110/4344 [23:26<1:01:57,  1.15s/it]

1 days 00:00:00
1 days 00:00:00


 26%|███████████████████████████████████████████████████████████████████████████████▏                                                                                                                                                                                                                           | 1150/4344 [24:27<1:17:02,  1.45s/it]

This model's maximum context length is 8193 tokens, however you requested 9555 tokens (8531 in your prompt; 1024 for the completion). Please reduce your prompt; or completion length.


 29%|███████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                     | 1260/4344 [26:09<47:35,  1.08it/s]

In [32]:
NNDemo = False
max_demo = 5
template = 'original-sql-py'
program = 'sql-py'
def func(i):
    codex_prompter = CodexAnswerCOTExecutor_BeamSeach(
                                              f'prompt_template/{template}.json',
                                              dataset.iloc[i]['id'], 
                                              dataset.iloc[i]['utterance'], 
                                              dataset.iloc[i]['context'], 
                                              dataset.iloc[i]['targetValue'],  
                                              base_path='./dataset/WikiTableQuestions/',
                                              demo_file=f'few-shot-demo/WikiTQ-{program}.json',
                                             )
    codex_prompter.max_demo = max_demo
    # codex_prompter._gen_gpt_prompt()
    codex_prompter._gen_gpt_prompt(NNDemo, ft)
    codex_prompter._get_gpt_prediction(beam_size=5)
    log = codex_prompter._log_dict()
    return log
a = func(0)

In [33]:
print(a)

{'id': 'nu-0', 'utterance': 'which country had the most cyclists finish within the top 10?', 'source_csv': './dataset/WikiTableQuestions/csv/203-csv/733.csv', 'target_value': 'Italy', 'predicted_value': 'ESP', 'prompt': 'The database table DF is shown as follows:\n[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss\n---\n[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18\n[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20\n[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14\n...\n[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108\n[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan\n\nAn

In [50]:
print(a['decision_prompts'][1])
# print(a['action_prompts'][0])

# print(a['gpt_original_output'][0])

The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or 

In [57]:
print(a['prompt'])

The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or 

In [51]:
print(a['prompt'])

The database table DF is shown as follows:
[HEAD]: name|c_1989|c_1990|c_1991|c_1992|c_1993|c_1994|c_1995|c_1996|c_1997|c_1998|c_1999|c_2000|c_2001|c_2002|c_2003|c_2004|c_2005|c_2006|c_2007|c_2008|c_2009|c_2010|career_sr|career_win_loss
---
[ROW] 1: Australian Open|A|A|1R|A|2R|3R|2R|1R|A|3R|4R|1R|2R|1R|3R|2R|1R|QF|3R|2R|3R|1R|0 / 18|22–18
[ROW] 2: French Open|1R|2R|4R|1R|1R|3R|1R|A|1R|3R|1R|2R|4R|2R|2R|3R|1R|1R|1R|2R|1R|A|0 / 20|17–20
[ROW] 3: Wimbledon|A|1R|A|A|A|A|1R|A|1R|A|2R|2R|3R|2R|2R|2R|2R|2R|2R|1R|2R|A|0 / 14|11–14
...
[ROW] 17: Annual Win-Loss|nan|2–4|7–5|3–5|6–4|2–1|5–4|2–1|12–6|10–9|10–7|12–9|13–9|9–9|2–7|8–5|7–7|3–8|4–3|2–3|1–2|0–0|nan|120–108
[ROW] 18: Year End Ranking|235|62|43|43|55|46|102|118|29|41|34|31|22|35|62|52|58|52|37|52|68|–|nan|nan

Answer the following question based on the data above: "did he win more at the australian open or indian wells?". Execute SQL or Python code step-by-step and finally answer the question. Choose from generating a SQL, Python code, or 